In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sqlite3

BASE_URL = "https://movie.naver.com/movie"


def get_page(page_url):
    """
    get_page 함수는 페이지 URL 을 받아 해당 페이지를 가져오고 파싱한 두
    결과들을 리턴합니다.

    예를 들어 `page_url` 이 `https://github.com` 으로 주어진다면
        1. 해당 페이지를 requests 라이브러리를 통해서 가져오고 해당 response 객체를 page 변수로 저장
        2. 1번의 response 의 html 을 BeautifulSoup 으로 파싱한 soup 객체를 soup 변수로 저장
        3. 저장한 soup, page 들을 리턴하고 함수 종료

    파라미터:
        - page_url: 받아올 페이지 url 정보입니다.

    리턴:
        - soup: BeautifulSoup 으로 파싱한 객체
        - page: requests 을 통해 받은 페이지 (requests 에서 사용하는 response
        객체입니다).
    """
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')



    return soup, page


def get_movie_code(movie_title):
    """
    get_movie_code 함수는 영화 제목을 받으면 해당 영화 제목으로 검색했을 때
    가장 먼저 나오는 영화의 아이디를 리턴합니다.

    해당 영화의 아이디는 네이버에서 지정한대로 사용합니다. 
    여기에서 네이버에서 지정한 아이디란 예를 들어 다음과 같습니다:
        - `https://movie.naver.com/` 에 접속
        - 검색란에 영화 제목 (예: Soul) 입력 뒤 검색
        - 해당 영화 페이지의 URL (예: `https://movie.naver.com/movie/bi/mi/basic.nhn?code=184517`) 의 'code=' 뒤에 나오는 숫자

    파라미터:
        - movie_title: 리뷰를 스크레이핑할 영화 제목이 담긴 문자열(str) 입니다.

    리턴:
        - 영화 아이디 번호: 네이버에서 지정한 영화의 아이디 번호가 담긴
        숫자(int) 입니다.
    """
    search_url = f"{BASE_URL}/search/result.naver?query={movie_title}&section=all&ie=utf8"
    movie_code = None
    soup, _ = get_page(search_url)   
    search_list=soup.find('ul',class_='search_list_1')
    movie_code= int(search_list.find('a')['href'].split('code=')[1])


    return movie_code

def get_url(movie_title):
    movie_code= get_movie_code(movie_title)
    return f"{BASE_URL}/point/af/list.naver?st=mcode&sword={movie_code}&target=after&page=1"


def get_reviews(movie_code, page_num):
    """
    get_reviews 함수는 리뷰들이 담긴 리뷰 리스트를 리턴해주는 함수입니다.

    각 리뷰는 다음과 같은 파이썬 딕셔너리 형태입니다:
        {
            'review_text': 리뷰 글이 담긴 문자열(str) 입니다,
            'review_star': 리뷰 별점이 담긴 숫자(int) 입니다
        }

    파라미터:
        - movie_code: 네이버에서 지정한 영화 아이디 번호가 담긴 숫자(int)
        입니다.
        - page_num: 리뷰를 몇 번째 리뷰 페이지에서 가져와야 하는지 담긴
        숫자(int) 입니다. 아무것도 주어지지 않은 경우 기본값은 1 입니다.

    리턴:
        - 리뷰 리스트: 스크레이핑한 리뷰들이 각각 파이썬 딕셔너리로 위에 명시된
        형태로 저장된 리스트입니다.
    """
    review_url = f"{BASE_URL}/point/af/list.naver?st=mcode&sword={movie_code}&target=after&page={page_num}"
    review_list = []
    soup, _ = get_page(review_url)   
    score_list=soup.find('table',class_='list_netizen').find('tbody').find_all('tr')
    
    text_list = []
    star_list = []
    for reple in score_list:
        text_list += [reple.find_all('td')[1].select_one('br').next_sibling.strip()]
        star_list += [int(reple.find('div',class_='list_netizen_score').find('em').text)]
        #review_dict['review_text'] = reple.find_all('td')[1].select_one('br').next_sibling.strip()
        #review_dict['review_star'] = int(reple.find('div',class_='list_netizen_score').find('em').text)
    
    return text_list,star_list



def scrape_by_page_num(movie_title,database_conn,database_cur, page_num=10):
    """
    scrape_by_page_num 함수는 페이지 수를 기준으로 리뷰를 스크레이핑하는
    함수입니다.

    파라미터:
        - movie_title: 리뷰를 스크레이핑할 영화 제목이 담긴 문자열(str) 입니다.
        - page_num: 첫 번째 페이지에서부터 스크레이핑할 페이지 개수가 담긴
        숫자(int) 입니다.

    리턴:
        - 리뷰 리스트: 주어진 page_num 만큼의 페이지에서부터 스크레이핑한
        리뷰를 담은 파이썬 리스트입니다. (각 리뷰 항목은 get_reviews 에서
        명시된 파이썬 딕셔너리 형태여야 합니다.)
    """
    try: 
        page = requests.get(get_url(movie_title))
        soup = BeautifulSoup(page.content, 'html.parser')
    
        if len(soup.find('div',class_='paging').find('div').find_all('a')) >10:
            get_reviews(get_movie_code(movie_title), 10)
            for page in range(1,page_num+1):
                text_list,star_list=get_reviews(get_movie_code(movie_title), page)
                cur.executemany('INSERT INTO Review(review_text,review_star,movie_title)VALUES(?,?,?)',list(zip(text_list,star_list,[movie_title]*len(text_list))))

        else:
            pass
    except:
        pass
    conn.commit()
    
    


In [ ]:
conn = sqlite3.connect('movie.db')
cur = conn.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS Review (
                        id INTEGER,
                        review_text TEXT,
                        review_star FLOAT,
                        movie_title VARCHAR(128),
                        PRIMARY KEY (id)
                        );""")


In [ ]:
for movie_title in list(service_movie['movieNm'].values):
    movie_review_df = scrape_by_page_num(movie_title,conn,cur)

In [ ]:
query =cur.execute('SELECT * From Review')
cols = [column[0] for column in query.description]
movie_df = pd.DataFrame(data = query.fetchall(), columns=cols)

In [ ]:
new_service=service_movie[service_movie['movieNm'].isin(movie_df['movie_title'].unique())]

In [ ]:
new_service.drop('Unnamed: 0',axis=1,inplace=True)

C:\Users\huise\AppData\Local\Temp\ipykernel_17692\2074088602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_service.drop('Unnamed: 0',axis=1,inplace=True)


In [ ]:
new_service.reset_index(drop=True, inplace=True)

In [ ]:
new_service.to_csv('서비스영화최종.csv')

In [ ]:
service_movi

In [ ]:
service_movie=pd.read_csv('서비스영화목록.csv')

In [ ]:
i

# Kobis  APi

In [ ]:
#
import os
import sys
import json
import requests
import pandas as pd
import sqlite3
key = "api키"
num=0
movie_df = pd.DataFrame(columns= ['movieNm', 'genreAlt', 'nationAlt','director','company'])
while True:
    num+=1
    url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key={key}&curPage={num}&itemPerPage=100" 
    try: 
        request = requests.get(url)
        movie = json.loads(request.content)
        start =0+100*(num-1)
        end = 100*num
        for idx in range(start,end):
            movie_df.loc[idx,'movieNm'] = movie['movieListResult']['movieList'][idx%100]['movieNm']
            movie_df.loc[idx,'prdtYear'] = movie['movieListResult']['movieList'][idx%100]['prdtYear']
            movie_df.loc[idx,'genreAlt'] = movie['movieListResult']['movieList'][idx%100]['genreAlt']
            movie_df.loc[idx,'nationAlt'] = movie['movieListResult']['movieList'][idx%100]['nationAlt']
            movie_df.loc[idx,'director'] = [director['peopleNm'] for director in movie['movieListResult']['movieList'][idx%100]['directors']]
            movie_df.loc[idx,'company'] = [company['companyNm'] for company in movie['movieListResult']['movieList'][idx%100]['companys']]
    except:
        break

In [ ]:
# 에로영화는 서비스 항목에서 제외할 것이기 때문에 삭제해준다.
# 장르가 따로 표기 되지 않은 영화는 제거해준다.

service_movie= movie_df[(-movie_df['genreAlt'].str.contains('에로')) & (movie_df['genreAlt']!='')]

In [ ]:
# 아무것도 반환되지 않은(Nan)데이터 빼기
for col in service_movie.columns:
    service_movie =service_movie[service_movie[cool].astype('bool') == True]


In [ ]:
# 데이터 베이스 저장
conn = sqlite3('movie.db')
cur = sqlite3('movie.db')
service_movie.to_csv('service_movie',conn)

# modeling위한 크롤링

In [1]:
# modeling위한 최근 댓글 10000개 크롤링
import random
import time

url = "https://movie.naver.com/movie/point/af/list.nhn?&page="
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36'}


score_list = []
comment_list = []


# Tag 꺼내기
for page in range(1,1000):
    web = requests.get(url+str(page), headers = headers).content
    soup = BeautifulSoup( web, 'html.parser')
    
    star_score_lst = soup.find_all('div',{'class':"list_netizen_score"})
    for star_score in star_score_lst:
        score_list.append(star_score.find('em').text)
        
    comment_lst = soup.find_all('td', {'class':"title"})
    for comment in comment_lst:

        # br class 다음 문자열을 불러옴. next_sibling
        comment_list.append(comment.select_one('br').next_sibling.strip())
    
    interval = round(random.uniform(0.2, 1.2),2)
    time.sleep(interval)
    
    if page%100 ==0:
        print(page)
        
cur.executemany('INSERT INTO Review_model(review_text,review_star,movie_title)VALUES(?,?)',list(zip(comment_list,score_list)))
conn.commit()

In [ ]:
query =cur.execute('SELECT * From Review_model')
cols = [column[0] for column in query.description]
df_model = pd.DataFrame(data = query.fetchall(), columns=cols)

In [ ]:
df_model

,id,review_text,review_star
0,1,스토리를 이해하면서 더욱 흥미롭게 감상했어요,8.0
1,2,같은 길을 걷는다고 생각했는데 만날수 없는 평행선이었다.,9.0
2,3,초반 5분은 시선을 확끄나 이어가질 못한다 좀 유치하고 이야기가 허술한 부분이 있지...,7.0
3,4,"적당함이 없네 ,보고 속이 내내 울렁거림. 비위 낮은 분들 절대로 보지마세여, ,",1.0
4,5,지루할틈이 없었어요. 넘 재미있게봤어요,10.0
...,...,...,...
9995,9996,서인국 조연이냐 머이리 빨리 뒤지냐,2.0
9996,9997,간만에 너무 재밌게 봣네요,10.0
9997,9998,,10.0
9998,9999,,10.0


# 모델링 위한 전처리

In [ ]:
# 텍스트 없이 별점만 남긴 경우 제거
df_model=df_model[df_model['review_text']!='']

In [ ]:
# 4,5,6점 데이터 제거, 10,9,8,7점 리뷰 - 긍정 1,2,3점 부정으로 분류
df_model=df_model[-df_model['review_star'].isin([4,5,6])]

In [ ]:
def neg_pos(x):
    if x >=7:
        return 1
    else:
        return 0

In [ ]:
df_model['positiveness'] = df_model['review_star'].apply(lambda x: neg_pos(x))

C:\Users\huise\AppData\Local\Temp\ipykernel_17692\2155036898.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['positiveness'] = df_model['review_star'].apply(lambda x: neg_pos(x))


In [ ]:
df_model

,review_text,review_star,positiveness
0,스토리를 이해하면서 더욱 흥미롭게 감상했어요,8.0,1
1,같은 길을 걷는다고 생각했는데 만날수 없는 평행선이었다.,9.0,1
2,초반 5분은 시선을 확끄나 이어가질 못한다 좀 유치하고 이야기가 허술한 부분이 있지...,7.0,1
3,"적당함이 없네 ,보고 속이 내내 울렁거림. 비위 낮은 분들 절대로 보지마세여, ,",1.0,-1
4,지루할틈이 없었어요. 넘 재미있게봤어요,10.0,1
...,...,...,...
8178,재밌는데 왜들 그러지 ㅋㅋ 잔인하긴함,10.0,1
8179,다큐형식의 전쟁영화 중에서도 긴장감을 놓지 않는다,7.0,1
8180,서인국 조연이냐 머이리 빨리 뒤지냐,2.0,-1
8181,간만에 너무 재밌게 봣네요,10.0,1


In [ ]:
df_model.drop('id',axis=1,inplace=True)
df_model.reset_index(drop=True,inplace=True)

C:\Users\huise\AppData\Local\Temp\ipykernel_17692\3078714020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.drop('id',axis=1,inplace=True)


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.8 MB/s 
     |████████████████████████████████| 465 kB 44.5 MB/s 


# 모델링

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm import tqdm
import re
import collections
from wordcloud import STOPWORDS
from scipy.sparse import csr_matrix
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from wordcloud import WordCloud
from konlpy.tag import Okt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve, auc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]') 
    result = hangul.sub('', text)  
    return result

In [2]:
#말뭉치 만들기
corpus = "".join(df_model['review_text'].tolist())
# 정규식 적용
corpus_accept = apply_regular_expression(corpus)


In [ ]:
raw_pos_tagged = Okt().pos(corpus, norm=True, stem=True)

In [ ]:
# set 함수를 사용하여 raw_pos_tagged pos값을 가져오고 중복은 제거한 순수한 pos값을 남긴다.
set_of_tag = set()
for tag in raw_pos_tagged:
    set_of_tag.add(tag[1])

In [ ]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stop_words = set([x[0] for x in stopwords])
movie_words = {'하다', '보다','있다','없다','너무','이다','영화',
           '되다','않다','같다','만들다','그냥','보고','정말',
           '가다','들다','진짜','싶다','정도','오다','많다',
           '연기','배우','그리고','부분','나다','편이','분들',
            '작품','영화','아니다','되는','겁니다','감독','합니다','싶을','같네'}
stop_words = stop_words.union(movie_words)

In [ ]:
word_cleaned = []

for word in raw_pos_tagged:
    if word[1] not in ["Josa", "Eomi", "Punctuation", "Foreign", "Number", "Hashtag", "URL","PreEomi"]: 
        if (len(word[0]) != 1) & (word[0] not in stop_words):
            word_cleaned.append(word[0])
        else :
            stop_words.add(word[0])
    
stop_words_lst = list(stop_words)
word_cleaned[:10]

['스토리', '이해', '더욱', '흥미롭다', '감상', '걷다', '생각', '날수', '평행선', '초반']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
def text_cleaning(text):
    hangul = re.compile('[^ a-z A-Z ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    words = okt.morphs(result, norm=True)
    words = [x for x in words if len(x) > 1]  # 한글자 키워드 제거
    words = [x for x in words if x not in stop_words_lst]  # 불용어 제거
    return words

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(df_model['review_text'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

C:\Users\huise\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
word_count_dict = dict(zip(word_list, count_list))
word_Freq = pd.DataFrame(word_count_dict.items())
word_Freq.tail(20)

,0,1
18927,힘들어지는거,1
18928,힘들어진거,1
18929,힘들어하고,1
18930,힘들어하는데,1
18931,힘들었고,1
18932,힘들었네요,1
18933,힘들었다,2
18934,힘들었던,2
18935,힘들었어도,1
18936,힘들었어요,2


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

In [ ]:
from sklearn.model_selection import train_test_split

x_data = tf_idf_vect
y_data = df_model['positiveness']
x_train, x_test, y_train, y_test, df_train, df_test = train_test_split(x_data, y_data, df_model, test_size = 0.3, random_state=0)
x_train.shape, y_train.shape

((5728, 18947), (5728,))

In [ ]:
lr =LogisticRegression(random_state=0)
lr.fit(x_train, y_train)

lr_pred = lr.predict(x_test)
print('accuracy: %.4f' % accuracy_score(y_test, lr_pred))

accuracy: 0.8460


In [ ]:
import pickle

with open('model.pkl','wb') as pickle_file:
    pickle.dump(lr, pickle_file)

In [ ]:
df_test['pred'] = lr_pred

# 네이버 영화 정화 크롤링

In [ ]:
import os
import sys
import json
import requests
import pandas as pd
import pickle
from google.colab import drive
drive.mount('/content/drive')
import sqlite3
conn = sqlite3.connect('/content/drive/MyDrive/섹션3 프로젝트/movie.db')
cur = conn.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS Movie_info(
                        id INTEGER,
                        movie_name TEXT,
                        image TEXT,
                        actor TEXT,
                        userRating Float,
                        PRIMARY KEY (id)
                        );""")

In [ ]:
import os
import sys
import urllib.request
import json
import time
import pandas as pd

js_list=[]


for movie_name in list(new_service['movieNm']):
    num +=1
    if num%10==0:
        time.sleep(2)
        print(num)
    client_id = "키"
    client_secret = "비밀키"
    encText = urllib.parse.quote(movie_name)
    url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    response_body = response.read()
    js_list+=[json.loads(response_body.decode('utf-8'))]

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

In [ ]:
with open('/content/drive/MyDrive/섹션3 프로젝트/data_list.pkl','rb') as pickle_file:
   js_list= pickle.load(pickle_file)

In [ ]:
import pickle
with open('/content/drive/MyDrive/섹션3 프로젝트/df_final.pkl','rb') as pickle_file:
   js_list= pickle.load(pickle_file)

In [ ]:
js_list

,word,word_num,score
0,스토리,9688,-0.634646
1,이해,13391,-0.059383
2,더욱,4130,0.362079
3,흥미롭게,18874,0.282715
4,감상,655,-0.152026
...,...,...,...
18942,하실거면,17813,-0.231878
18943,다니시길,3644,-0.231878
18944,잔인하긴,14015,0.142815
18945,뒤지냐,4667,-0.191163


In [ ]:
movie_info=pd.DataFrame(columns=js_list[0]['items'][0].keys())

In [ ]:
movie_info.drop('subtitle',axis=1, inplace=True)

In [ ]:
movie_info['title']= new_service['movieNm']

In [ ]:
def director_engineering(l):
    for ind,d_name in enumerate(l):
        l[ind]=d_name.lstrip("[").rstrip("]").strip().strip("'")
    return l

In [ ]:
idx=-1
for movie_json in js_list:
    idx+=1
    if movie_json['display']==1:
        movie_info.loc[idx,'link'] = movie_json['items'][0]['link']
        movie_info.loc[idx,'image'] = movie_json['items'][0]['image']
        movie_info.loc[idx,'pubDate'] = movie_json['items'][0]['pubDate']
        movie_info.loc[idx,'director'] = movie_json['items'][0]['director']
        movie_info.loc[idx,'actor'] = movie_json['items'][0]['actor']
        movie_info.loc[idx,'userRating'] = movie_json['items'][0]['userRating']
    else:
        for multi_json in movie_json['items']:
            try:
                if (int(new_service.loc[idx,'prdtYear']) == int(multi_json['pubDate'])) & (multi_json['director'].split('|')[0] in new_service.loc[idx,['director']].str.split(',').apply(lambda x:director_engineering(x))['director']):
                        movie_info.loc[idx,'link'] = multi_json['link']
                        movie_info.loc[idx,'image'] = multi_json['image']
                        movie_info.loc[idx,'pubDate'] = multi_json['pubDate']
                        movie_info.loc[idx,'director'] = multi_json['director']
                        movie_info.loc[idx,'actor'] = multi_json['actor']
                        movie_info.loc[idx,'userRating'] = multi_json['userRating']
                        break
            except:
                pass

    
        
        
        

In [ ]:
movie_info[movie_info['link'].isnull()].index

Int64Index([   5,   30,   31,   39,   48,   50,   70,   90,   93,  107,
            ...
            2928, 2942, 2945, 2946, 2949, 2958, 2961, 2964, 2971, 2972],
           dtype='int64', length=363)

In [ ]:
movie_info.loc[movie_info[movie_info['link'].isnull()].index,'title']

5                       리멤버
30      섹스, 거짓말 그리고 비디오 테이프
31                     2046
39                    인크레더블
48                  나쁜 녀석들 
               ...         
2958                 1 On 1
2961                   동방불패
2964                  드라큐라 
2971                 블랙 위도우
2972                    스텔라
Name: title, Length: 363, dtype: object

In [ ]:
import os
import sys
import urllib.request
import json
import time
import pandas as pd

js_list=[]

num = 0

for movie_name in movie_info.loc[movie_info[movie_info['link'].isnull()].index,'title']:
    num +=1
    if num%10==0:
        time.sleep(2)
        print(num)
    client_id = "RObdvWLnwy5exMUU5rYq"
    client_secret = "suKykfzQiD"
    encText = urllib.parse.quote(movie_name)
    url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText+"&display=100"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    response_body = response.read()
    js_list+=[json.loads(response_body.decode('utf-8'))]

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360


In [ ]:
for movie_json,idx in list(zip(js_list,list(movie_info[movie_info['link'].isnull()].index))):
    if movie_json['display']==1:
        movie_info.loc[idx,'link'] = movie_json['items'][0]['link']
        movie_info.loc[idx,'image'] = movie_json['items'][0]['image']
        movie_info.loc[idx,'pubDate'] = movie_json['items'][0]['pubDate']
        movie_info.loc[idx,'director'] = movie_json['items'][0]['director']
        movie_info.loc[idx,'actor'] = movie_json['items'][0]['actor']
        movie_info.loc[idx,'userRating'] = movie_json['items'][0]['userRating']
    else:
        for multi_json in movie_json['items']:
            try:
                if (int(new_service.loc[idx,'prdtYear']) == int(multi_json['pubDate'])) & (multi_json['director'].split('|')[0] in new_service.loc[idx,['director']].str.split(',').apply(lambda x:director_engineering(x))['director']):
                        movie_info.loc[idx,'link'] = multi_json['link']
                        movie_info.loc[idx,'image'] = multi_json['image']
                        movie_info.loc[idx,'pubDate'] = multi_json['pubDate']
                        movie_info.loc[idx,'director'] = multi_json['director']
                        movie_info.loc[idx,'actor'] = multi_json['actor']
                        movie_info.loc[idx,'userRating'] = multi_json['userRating']
                        break
            except:
                pass

    
        
        
        

In [ ]:
movie_info_clean= movie_info.dropna()

In [ ]:
new_service[new_service['movieNm'].isin(list(movie_info_clean['title']))]

,Unnamed: 0,movieNm,genreAlt,nationAlt,director,company,prdtYear
0,0,정직한 후보2,['코미디'],['한국'],['장유정'],"['(주)수필름', '(주)홍필름']",2022.0
1,1,인생은 아름다워,['뮤지컬'],['한국'],['최국희'],['더램프(주)'],2020.0
2,2,거래완료,['드라마'],['한국'],['조경호'],['한국예술종합학교 산학협력단'],2020.0
3,3,대무가,"['드라마', '코미디']",['한국'],['이한종'],['(주)쿠키픽쳐스'],2022.0
4,4,자백,"['범죄', '스릴러']",['한국'],['윤종석'],['리얼라이즈픽쳐스(주)'],2020.0
...,...,...,...,...,...,...,...
2979,2979,컬러퍼플,['드라마'],['미국'],['스티븐 스필버그'],"['엠블린엔터테인먼트', '구버피터스컴퍼니', '워너 브러더스 픽쳐스']",1985.0
2980,2980,킬링필드,"['드라마', '사극', '전쟁']",['영국'],['롤랑 조페'],['워너 브러더스 픽쳐스'],1984.0
2981,2981,티라노의 발톱,"['SF', '액션', '코미디']",['한국'],['심형래'],['영구아트무비'],1994.0
2982,2982,007 살인면허,"['드라마', '범죄', '스릴러', '액션']",['미국'],['존 글렌'],"['다냐크 프로덕션', '이온 프로덕션']",1989.0


In [ ]:
movie_info_clean[['genre','nation']]= new_service[new_service['movieNm'].isin(list(movie_info_clean['title']))][['genreAlt','nationAlt']]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
movie_info_clean.to_csv('/content/drive/MyDrive/섹션3 프로젝트/final_movie.csv')

In [ ]:
movie_info_clean['link'][0]

'https://movie.naver.com/movie/bi/mi/basic.nhn?code=210852'

In [ ]:
import sqlite3
conn = sqlite3.connect('/content/drive/MyDrive/섹션3 프로젝트/pjmovie.db')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS Review')
cur.execute("""CREATE TABLE IF NOT EXISTS Review (
                        id INTEGER,
                        review_text TEXT,
                        review_star FLOAT,
                        movie_title VARCHAR(128),
                        PRIMARY KEY (id)
                        );""")

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

BASE_URL = "https://movie.naver.com/movie"


def get_page(page_url):
    """
    get_page 함수는 페이지 URL 을 받아 해당 페이지를 가져오고 파싱한 두
    결과들을 리턴합니다.

    예를 들어 `page_url` 이 `https://github.com` 으로 주어진다면
        1. 해당 페이지를 requests 라이브러리를 통해서 가져오고 해당 response 객체를 page 변수로 저장
        2. 1번의 response 의 html 을 BeautifulSoup 으로 파싱한 soup 객체를 soup 변수로 저장
        3. 저장한 soup, page 들을 리턴하고 함수 종료

    파라미터:
        - page_url: 받아올 페이지 url 정보입니다.

    리턴:
        - soup: BeautifulSoup 으로 파싱한 객체
        - page: requests 을 통해 받은 페이지 (requests 에서 사용하는 response
        객체입니다).
    """
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')



    return soup, page


def get_movie_code(movie_link):
    """
    get_movie_code 함수는 영화 제목을 받으면 해당 영화 제목으로 검색했을 때
    가장 먼저 나오는 영화의 아이디를 리턴합니다.

    해당 영화의 아이디는 네이버에서 지정한대로 사용합니다. 
    여기에서 네이버에서 지정한 아이디란 예를 들어 다음과 같습니다:
        - `https://movie.naver.com/` 에 접속
        - 검색란에 영화 제목 (예: Soul) 입력 뒤 검색
        - 해당 영화 페이지의 URL (예: `https://movie.naver.com/movie/bi/mi/basic.nhn?code=184517`) 의 'code=' 뒤에 나오는 숫자

    파라미터:
        - movie_title: 리뷰를 스크레이핑할 영화 제목이 담긴 문자열(str) 입니다.

    리턴:
        - 영화 아이디 번호: 네이버에서 지정한 영화의 아이디 번호가 담긴
        숫자(int) 입니다.
    """
    
    movie_code= movie_link.split('code=')[1]
    return movie_code

def get_url(movie_link):
    movie_code=get_movie_code(movie_link)
    return f"{BASE_URL}/point/af/list.naver?st=mcode&sword={movie_code}&target=after&page=1"


def get_reviews(movie_code, page_num):
    """
    get_reviews 함수는 리뷰들이 담긴 리뷰 리스트를 리턴해주는 함수입니다.

    각 리뷰는 다음과 같은 파이썬 딕셔너리 형태입니다:
        {
            'review_text': 리뷰 글이 담긴 문자열(str) 입니다,
            'review_star': 리뷰 별점이 담긴 숫자(int) 입니다
        }

    파라미터:
        - movie_code: 네이버에서 지정한 영화 아이디 번호가 담긴 숫자(int)
        입니다.
        - page_num: 리뷰를 몇 번째 리뷰 페이지에서 가져와야 하는지 담긴
        숫자(int) 입니다. 아무것도 주어지지 않은 경우 기본값은 1 입니다.

    리턴:
        - 리뷰 리스트: 스크레이핑한 리뷰들이 각각 파이썬 딕셔너리로 위에 명시된
        형태로 저장된 리스트입니다.
    """
    review_url = f"{BASE_URL}/point/af/list.naver?st=mcode&sword={movie_code}&target=after&page={page_num}"
    review_list = []
    soup, _ = get_page(review_url)   
    score_list=soup.find('table',class_='list_netizen').find('tbody').find_all('tr')
    
    text_list = []
    star_list = []
    for reple in score_list:
        text_list += [reple.find_all('td')[1].select_one('br').next_sibling.strip()]
        star_list += [int(reple.find('div',class_='list_netizen_score').find('em').text)]
        
    
    return text_list,star_list



def scrape_by_page_num(movie_title,movie_link,database_conn,database_cur, page_num=10):
    """
    scrape_by_page_num 함수는 페이지 수를 기준으로 리뷰를 스크레이핑하는
    함수입니다.

    파라미터:
        - movie_title: 리뷰를 스크레이핑할 영화 제목이 담긴 문자열(str) 입니다.
        - page_num: 첫 번째 페이지에서부터 스크레이핑할 페이지 개수가 담긴
        숫자(int) 입니다.

    리턴:
        - 리뷰 리스트: 주어진 page_num 만큼의 페이지에서부터 스크레이핑한
        리뷰를 담은 파이썬 리스트입니다. (각 리뷰 항목은 get_reviews 에서
        명시된 파이썬 딕셔너리 형태여야 합니다.)
    """
    try: 
        page = requests.get(get_url(movie_link))
        soup = BeautifulSoup(page.content, 'html.parser')
    
        if len(soup.find('div',class_='paging').find('div').find_all('a')) >10:
            get_reviews(get_movie_code(movie_link), 10)
            for page in range(1,page_num+1):
                text_list,star_list=get_reviews(get_movie_code(movie_link), page)
                cur.executemany('INSERT INTO Review(review_text,review_star,movie_title)VALUES(?,?,?)',list(zip(text_list,star_list,[movie_title]*len(text_list))))

        else:
            pass
    except:
        pass
    conn.commit()
    




In [ ]:
movie_info_clean.info

In [ ]:
num=0
for movie_title ,movie_link in list(zip(list(movie_info_clean['title'].values),list(movie_info_clean['link'].values))):
    num+=1
    movie_review_df = scrape_by_page_num(movie_title,movie_link,conn,cur)
    if num%50==0:
        print(num)

50


In [ ]:
cur.executemany('INSERT INTO Movie_info(movie_name,image,actor,userRating)VALUES(?,?,?,?)',list(zip(movie_list,image_list,actor_list,userRating_list)))
conn.commit()